# W2V-BERT Embedding Demo (Packaged)

This notebook demonstrates how to load the packaged (eager) W2V-BERT speaker model and run inference to produce embeddings. The repository has been standardized so notebooks use fixed, repo-relative artifact and dataset locations by default:

- Eager artifacts: `../pretrained/w2vbert_speaker_eager`
- Scripted artifacts: `../pretrained/w2vbert_speaker_scripted`
- Dataset used in the examples: `../datasets/voxceleb1test`

See `ARTIFACTS.md` at the repository root for what to download, expected files in each directory, and examples of how to override locations if you store artifacts elsewhere.


In [7]:
from pathlib import Path

def find_repo_root(start: Path) -> Path:
    for candidate in [start, *start.parents]:
        if (candidate / "recipes").exists() and (candidate / "deeplab").exists():
            return candidate
    raise RuntimeError(f"Unable to locate the repository root from {start}.")

NOTEBOOK_DIR = Path.cwd()
REPO_ROOT = find_repo_root(NOTEBOOK_DIR)

print(f"Notebook directory: {NOTEBOOK_DIR}")
print(f"Resolved repository root: {REPO_ROOT}")

Notebook directory: /Users/zb/NWG/w2v-BERT-2.0_SV/recipes/DeepASV/notebooks
Resolved repository root: /Users/zb/NWG/w2v-BERT-2.0_SV


In [ ]:
import torch
from pathlib import Path
from w2vbert_speaker import W2VBERT_SPK_Module

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use fixed, repo-relative locations for artifacts and data (per project convention)
# EAGER artifacts directory contains eager packaged model & checkpoint
EAGER_DIR = (REPO_ROOT.parent / "pretrained" / "w2vbert_speaker_eager").resolve()
# SCRIPTED artifacts directory contains the exported TorchScript and feature-extractor
SCRIPTED_DIR = (REPO_ROOT.parent / "pretrained" / "w2vbert_speaker_scripted").resolve()

checkpoint_path = (EAGER_DIR / "model_lmft_0.14.pth").resolve()
model_dir = EAGER_DIR

print(f"Using checkpoint: {checkpoint_path}")
print(f"Using eager model dir: {model_dir}")
print(f"Using scripted artifacts dir: {SCRIPTED_DIR}")

embedding_model = W2VBERT_SPK_Module(device=DEVICE, model_path=str(model_dir)).load_model(checkpoint_path)
spk_model = embedding_model.modules_dict["spk_model"]
target_sr = spk_model.front.feature_extractor.sampling_rate
print(f"Using device: {DEVICE}, target sample rate: {target_sr}")

Using checkpoint: /Users/zb/NWG/pretrained/audio2vector/ckpts/facebook/w2v-bert-2.0/model_lmft_0.14.pth
Using local encoder weights from: /Users/zb/NWG/pretrained/audio2vector/ckpts/facebook/w2v-bert-2.0
Using device: cpu, target sample rate: 16000


In [ ]:
import soundfile as sf
import torchaudio

# Standardized dataset directory
DATASET_DIR = (REPO_ROOT.parent / "datasets" / "voxceleb1test").resolve()

def load_waveform(path: Path, target_sr: int) -> tuple[torch.Tensor, int]:
    signal, sr = sf.read(str(path), dtype="float32")
    if signal.ndim > 1:
        signal = signal.mean(axis=1)
    waveform = torch.from_numpy(signal)
    if sr != target_sr:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
        waveform = resampler(waveform.unsqueeze(0)).squeeze(0)
        sr = target_sr
    waveform = waveform.unsqueeze(0).to(torch.float32)
    return waveform, sr

# Prefer a canonical example path under DATASET_DIR; update if you want a different sample
AUDIO_PATH = (DATASET_DIR / "wav" / "id10270" / "5r0dWxy17C8" / "00001.wav").resolve()
waveform, sr = load_waveform(AUDIO_PATH, target_sr)
print(f"Using audio file: {AUDIO_PATH}")
print(f"Loaded waveform shape: {waveform.shape}, sample rate: {sr}")

Using audio file: /Users/zb/NWG/datasets/voxceleb1test/wav/id10270/5r0dWxy17C8/00001.wav
Loaded waveform shape: torch.Size([1, 133761]), sample rate: 16000


In [13]:
with torch.inference_mode():
    embeddings = embedding_model(waveform.to(DEVICE))
embedding_vector = embeddings.squeeze(0).detach().cpu().numpy()

print(f"Embedding shape: {embedding_vector.shape}")
embedding_vector[:8]

Embedding shape: (256,)


array([-0.16015907, -0.8298738 ,  0.70560724, -0.14280552,  0.35778913,
        0.47867072, -0.12521656, -0.90179497], dtype=float32)

In [ ]:
from pathlib import Path
import torch.nn.functional as F

artifact_waveform = (SCRIPTED_DIR / "w2vbert_speaker_script.pt").resolve()
artifact_preprocessed = (SCRIPTED_DIR / "w2vbert_speaker_script_preprocessed.pt").resolve()

metadata_waveform = {"sample_rate": "", "embedding_dim": ""}
scripted_waveform = torch.jit.load(str(artifact_waveform), map_location="cpu", _extra_files=metadata_waveform)
metadata_preprocessed = {"sample_rate": "", "embedding_dim": "", "preprocessed": ""}
scripted_preprocessed = torch.jit.load(str(artifact_preprocessed), map_location="cpu", _extra_files=metadata_preprocessed)

with torch.inference_mode():
    waveform_scripted = waveform.cpu()
    emb_waveform = scripted_waveform(waveform_scripted).squeeze(0).cpu()

features = spk_model.front.feature_extractor(
    waveform_scripted.squeeze(0).cpu().numpy(),
    sampling_rate=target_sr,
    return_tensors="pt",
    padding=False,
    truncation=False,
    return_attention_mask=False,
)
input_features = features["input_features"].to(torch.float32)

with torch.inference_mode():
    emb_preprocessed = scripted_preprocessed(input_features).squeeze(0).cpu()

cos_waveform = F.cosine_similarity(torch.from_numpy(embedding_vector).unsqueeze(0), emb_waveform.unsqueeze(0)).item()
l2_waveform = torch.norm(torch.from_numpy(embedding_vector) - emb_waveform).item()
cos_preprocessed = F.cosine_similarity(torch.from_numpy(embedding_vector).unsqueeze(0), emb_preprocessed.unsqueeze(0)).item()
l2_preprocessed = torch.norm(torch.from_numpy(embedding_vector) - emb_preprocessed).item()

print(f"cosine(eager, waveform_script) = {cos_waveform:.6f}")
print(f"    L2(eager, waveform_script) = {l2_waveform:.6e}")
print(f"cosine(eager, preprocessed_script) = {cos_preprocessed:.6f}")
print(f"    L2(eager, preprocessed_script) = {l2_preprocessed:.6e}")

In [ ]:
tolerance = 1e-5
if l2_preprocessed > tolerance:
    raise AssertionError(f"Preprocessed TorchScript deviates from eager more than {tolerance}: L2={l2_preprocessed}")
if l2_waveform > 1e-3:
    print(f"Warning: waveform TorchScript deviates from eager (L2={l2_waveform:.6e}). Use the preprocessed artifact for parity.")
else:
    print("Waveform TorchScript within tolerance of eager model.")